# Visualisation de données multidimensionnelles et réduction de variables

Nous allons effectuer une analyse des données d'expression génique dans le cancer du poumon en utilisant des méthodes d'apprentissage automatique non supervisées : l'Analyse en Composantes Principales (ACP) et le t-SNE.

Ces techniques permettent de projeter des données multidimensionnelles en 2 ou 3 dimensions afin de les visualiser plus facilement. Elles sont également utiles pour réduire le nombre de variables, éliminer les corrélations et atténuer le bruit.

# 1. Importer les données

Les données sont stockées dans le fichier lung_cancer.csv.

In [ ]:
import pandas as pd

data = pd.read_csv('lung_cancer.csv', sep=';', index_col='id_sample')
print('data', data.shape)

In [ ]:
data.head()

# Analyse de données

Pour examiner la distribution des échantillons dans chaque sous-type moléculaire, nous utilisons la méthode groupby de Pandas.

In [ ]:
data.groupby(['class']).size()

In [ ]:
data.groupby(['class']).size().plot(kind = "bar")

On observe que les données ne sont pas réparties de manière homogène en fonction du type de cancer.

# 2. Séparer les données d’expression et les étiquettes

Pour faciliter les calculs sur les données d'expression, il est préférable de séparer les valeurs numériques (expressions géniques) des étiquettes des sous-types moléculaires (class) en deux objets distincts.

Conformément aux conventions de scikit-learn, nous nommons la matrice de données d'expression X et les étiquettes associées y.

In [ ]:
X = data.select_dtypes('number') 
print('X', X.shape)

y = data['class']
print('y', y.shape)

# 3. Afficher les valeurs d'expression

Le code suivant permet d'afficher les valeurs d'expression de chaque gène, triées par moyenne.

In [ ]:
sort_by_mean = X.mean().sort_values(ascending=True)
X[sort_by_mean.index].plot(kind='box', figsize=(15, 4), rot=90, ylabel='Expression')

Certains gènes présentent une grande variation d'expression, tandis que d'autres sont plus stables. Ces différences de variance peuvent fortement influencer l'ACP, car cette méthode repose essentiellement sur la variance des données.
Par conséquent, il est indispensable de normaliser (standardiser) les données avant de réaliser l'ACP.

# 4. Appliquer une normalisation centrée-réduite aux données 

La normalisation centrée-réduite consiste à soustraire la moyenne et à diviser les valeurs d'expression par l'écart-type. Cela peut être réalisé avec l'objet StandardScaler de la bibliothèque scikit-learn.

In [9]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler() 
X_scaled = scaler.fit_transform(X) 
X_scaled = pd.DataFrame(X_scaled, index=X.index, columns=X.columns) 

Après normalisation, les valeurs d'expression sont ramenées à la même échelle : chaque gène a une moyenne égale à 0 et un écart-type égal à 1.

In [ ]:
X_scaled.plot(kind='box', figsize=(15, 4), rot=90, ylabel='Expression')

# 5. Faire une analyse en composantes principales (ACP)

Le calcul de l'ACP peut se faire à l'aide de l'objet `PCA` de `scikit-learn`.

In [11]:
from sklearn.decomposition import PCA

## 5.1  Calcul de l'ACP

Pour calculer une ACP, on utilise les données standardisées **X_scaled**.

In [12]:
pca = PCA() 
X_pca = pca.fit_transform(X_scaled) 

Pour faciliter l'exploitation des résultats, ceux-ci sont convertis en DataFrame Pandas avec des colonnes nommées de PC1 à PCn.

In [ ]:
pca_columns = ['PC' + str(c) for c in range(1, X_pca.shape[1]+1, 1)] 
X_pca = pd.DataFrame(X_pca, index=X.index, columns=pca_columns) 
X_pca.head()

## 5.2 Calcul de la variance expliquée

Les variances expliquées par chaque composante peuvent être converties en un objet Series de Pandas et présentées sous forme de pourcentage.

In [ ]:
explained_variance = pd.Series(dict(zip(X_pca.columns, 100.0*pca.explained_variance_ratio_)))
print(explained_variance.head())

In [ ]:
explained_variance.plot(kind='bar', figsize=(15, 4), rot=90, ylabel='Explained variance')

## 5.3 Visualisation des deux premières composantes principales de l’ACP

In [ ]:
X_pca.plot(x='PC1', y='PC2', kind='scatter', figsize=(5, 5), color='gray')

Les points se regroupent naturellement en clusters distincts, avec deux regroupements particulièrement visibles. Nous utilisons différentes couleurs pour chaque sous-type moléculaire afin de mieux interpréter ces regroupements.

In [ ]:
dict_colors = {'ADK': 'forestgreen', 'NTL': 'royalblue', 'SQC': 'orange'} 
y_colors = [dict_colors[yi] for yi in y]
X_pca.plot(x='PC1', y='PC2', kind='scatter', figsize=(5, 5), color=y_colors)

## 5.4 Visualisation des trois premières composantes principales de l’ACP

In [23]:
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot(projection='3d')
ax.scatter(X_pca['PC1'], X_pca['PC2'], X_pca['PC3'], marker='o', s=30, edgecolor='k', facecolor=y_colors)
ax.set_xlabel('PC1 - ' + '{:.1f}%'.format(explained_variance['PC1']))
ax.set_ylabel('PC2 - ' + '{:.1f}%'.format(explained_variance['PC2']))
ax.set_zlabel('PC3 - ' + '{:.1f}%'.format(explained_variance['PC3']))
ax.view_init(elev=15, azim=45)

# 6. Visualiser les données avec la méthode t-SNE

In [25]:
from sklearn.manifold import TSNE

## 6.1 Projection 2D

In [26]:
tsne = TSNE(n_components=2, init='pca', random_state=0, n_jobs=-1)
X_tsne = tsne.fit_transform(X_scaled)

In [ ]:
columns = ['DIM' + str(c) for c in range(1, X_tsne.shape[1]+1, 1)]
X_tsne = pd.DataFrame(X_tsne, index=X.index, columns=columns)
X_tsne.head()

In [ ]:
X_tsne.plot(x='DIM1', y='DIM2', kind='scatter', figsize=(5, 5), color=y_colors)

In [ ]:
# La divergence de Kullback-Leibler après optimisation
tsne.kl_divergence_

## 6.2 Projection 3D

In [30]:
tsne = TSNE(n_components=3, init='pca', perplexity=45, random_state=0, n_jobs=-1)
X_tsne = tsne.fit_transform(X_scaled)

In [ ]:
columns = ['DIM' + str(c) for c in range(1, X_tsne.shape[1]+1, 1)]
X_tsne = pd.DataFrame(X_tsne, index=X.index, columns=columns)
X_tsne.head()

In [ ]:
fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot(projection='3d')
ax.scatter(X_tsne['DIM1'], X_tsne['DIM2'], X_tsne['DIM3'], marker='o', s=30, edgecolor='k', facecolor=y_colors)
ax.set_xlabel('DIM1')
ax.set_ylabel('DIM2')
ax.set_zlabel('DIM3')
ax.view_init(elev=15, azim=45)

In [ ]:
tsne.kl_divergence_